## Eliminate Duplicates

In [3]:
from pandas import concat, DataFrame
from viedoors import FileMerger


def keep_n_duplicates(dataframe, n):

        duplicates = dataframe[dataframe.duplicated(keep=False)]
        keep_rows = duplicates.iloc[:n]
        cleaned_df = concat(
                [
                        dataframe[~dataframe.duplicated(keep=False)],
                        keep_rows
                ]
        ).reset_index(drop=True)
        return cleaned_df

def get_n(dataframe, aks):
        return len(dataframe.loc[dataframe["merge"]==aks])

In [4]:
cad = DataFrame({
    "merge": ["1234", "1234", "1234", "9876"],
    "value_01": [1, 1, 3, 4],
    "value_02": [5, 5, 7, 8]
})

npa = DataFrame({
    "merge": ["1234", "1234", "9876"],
    "value_03": [9, 9, 11],
    "value_04": [12, 12, None]
})

In [5]:
fm = FileMerger([cad, npa])
merge = fm.data_merge

In [6]:
merge.loc[~merge.duplicated()]

,merge,value_01,value_02,value_03,value_04
0,1234,1,5,9,12.0
4,1234,3,7,9,12.0
6,9876,4,8,11,NaN


Zwischenfazit: Die `420-00B1234-01` landet vier mal im Merge. Das liegt daran, dass sie **zweimal im CAD-File** und **zweimal im NPA-File** vorkam. Wir wollen sie aber nur so oft im Merge behalten, wie sie im CAD-File vorkam - also zweimal. Das Problem: Wenn wir einfach nur alle Voll-Duplikate (gleiche Werte in allen Zellen) löschen, dann bleibt nur ein Datensatz übrig. Wir wollen aber dass `n` Duplikate übrig bleiben - so viele, wie es Einträge im CAD-File gab.

In [7]:
new_merge = None
first = True

for aks in merge["merge"].unique():
    sub_df = merge.loc[merge["merge"]==aks]
    duplicate_mask = sub_df.duplicated()
    a = sub_df.loc[~duplicate_mask]

    for i in a.index:
        b = a.loc[a.index==i][list(cad.columns)]
        n = (cad == b.iloc[0]).all(axis=1).sum()

        if first:
            new_merge = DataFrame([a.iloc[0]] * n, columns=a.columns)
            first = False
        else:
            new_merge = concat([new_merge, a])

new_merge = new_merge.reset_index()
new_merge.drop("index", axis=1, inplace=True)
new_merge

,merge,value_01,value_02,value_03,value_04
0,1234,1,5,9,12.0
1,1234,1,5,9,12.0
2,1234,1,5,9,12.0
3,1234,3,7,9,12.0
4,9876,4,8,11,NaN


## Duplicate Overview